### Netflix ELT Project

### ETL stands for "Extract, Load, Transform"

#### Extract

The extract phase is the first step in the ELT process. In this phase, data is collected from various sources. These sources could be databases, CRM systems, social media platforms, or any other place where data is stored. The extracted data is often raw and unstructured and may come in various formats such as text, images, audio, or video.

#### Load

After the data is extracted, it’s loaded into a data storage system in the load phase. This system can be a database, a data warehouse, or a data lake.
The data is loaded as-is, without any transformation. This means that the data maintains its original format and structure, which can be beneficial for preserving the data’s original context and meaning.

#### Transform

During this phase, the data is prepared for analysis. This preparation can involve various operations such as cleaning, filtering, aggregating, and summarizing the data. The goal of the transformation is to convert the raw data into a format that’s easy to analyze and interpret.

In [ ]:
# Import the required libraries.

# For data extraction and transformation.
import pandas as pd 

# Connecting to DB, Loading data to destination.
import sqlalchemy as sql

In [ ]:
# Download the data from: https://www.kaggle.com/datasets/shivamb/netflix-shows
# Load the data using Pandas read_csv method.

df = pd.read_csv('netflix_titles.csv')

In [ ]:
# Connecting to our destination DB like SQL Server, PostgreSQL, MySQL...

engine = sql.create_engine('mssql://DB Connection Details')
conn=engine.connect()

In [ ]:
# Load the data to DB, if_exists = 'replace' will replace the existing data in table "netflix_raw"

df.to_sql('netflix_raw', con=conn , index=False, if_exists = 'replace')
conn.close()

In [ ]:
# Once data is loaded into DB, check if all data is loaded.

len(df)

In [ ]:
# No. of rows loaded is same. 
# But we notice an issue with the TITLE column, that some of the title is present as "?????" in DB.


df[df.show_id=='s5023']

In [ ]:
# Here we see that the Title contains a foreign language which is not properly displayed in DB.

### Issue

By default, if_exists = 'replace' is using varchar(max) for all the rows.
- To overcome this we will use nvarchar as data type for columns which has special characters.
- We will check the length of the data in each column and use that to replace with varchar(max).


In [ ]:
df.columns

In [ ]:
# Here we are looping over each column to get the max length of the column. 
# We are using Try/Except to catch the columns with different datatype and cast it to str.


for i in df.columns:
    try: 
        print(max(df[i].dropna().str.len()))
    except:
        print(len(str(df[i])))
        continue

In [ ]:
# Now we can use these lenght values to create a new table in DB using nvarchar(length)

# Once the new table is created, load the data again using if_exists = 'append' 

engine = sql.create_engine('mssql://DB Connection Details')
conn=engine.connect()
df.to_sql('netflix__raw', con=conn , index=False, if_exists = 'append')
conn.close()

In [ ]:
# Check how many NULL values are there in each Columns

df.isna()

In [ ]:
df.isna().sum()

Data Transformation and Cleaning is done using SQL. Refer to the SQL file.